### Adjust Word Group Repeat With Youtube Link

In [53]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [54]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [55]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

# file extention
file_ext = "Threegram"

In [56]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [57]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 2/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [58]:
path_folder_file = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 2/Result/2-Find Word Group Youtube Link/Turkish_Threegram_With_1000_Word_Selected_Count_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [61]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url,count
0,bir şey yok,134,136,kabul etmekten daha normal de bir şey yok,ySHVEb0jLFc,https://www.youtube.com/watch?v=ySHVEb0jLFc&t=...,7
1,bir şey yok,508,510,öyle birden bire damdan düşer gibi usta olmak ...,yUR3RBa1xP0,https://www.youtube.com/watch?v=yUR3RBa1xP0&t=...,7
2,bir şey yok,385,386,yine burada beklenmedik bir şey yok,jMtqToOsO90,https://www.youtube.com/watch?v=jMtqToOsO90&t=...,7
3,bir şey yok,562,564,kameranın üzerindeki mikrofondan gelen ses diy...,sEhRqPshDc4,https://www.youtube.com/watch?v=sEhRqPshDc4&t=...,7
4,bir şey yok,476,478,yapılabilecek daha akla yakın bir şey yok gibi...,_22z2rwaQ3o,https://www.youtube.com/watch?v=_22z2rwaQ3o&t=...,7
...,...,...,...,...,...,...,...
161,kendine iyi bak,856,859,sağ ol sen de kendine iyi bak i nşallah her şe...,HeKwkn3ByP0,https://www.youtube.com/watch?v=HeKwkn3ByP0&t=...,4
162,burada ne yapıyorsun,6428,6430,ve sen bana şöyle bir şey sormuştun sen burada...,tYBwrXTUMDQ,https://www.youtube.com/watch?v=tYBwrXTUMDQ&t=...,4
163,burada ne yapıyorsun,517,519,burada ne yapıyorsun kaç kişiyle çalışıyorsun,JwCmO6nFfLs,https://www.youtube.com/watch?v=JwCmO6nFfLs&t=...,4
164,burada ne yapıyorsun,621,623,burada ne yapıyorsun sesini ve ekranla ilgili ...,x33WWMw0Kb0,https://www.youtube.com/watch?v=x33WWMw0Kb0&t=...,4


In [62]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

28

In [63]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [64]:
## All word group result convert to join result in one row
#df_result_repeat = pd.DataFrame()
#for word_group in search_list:
#    # for repeat 1
#    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
#    # for repeat 2
#    try:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
#    except:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
#    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
#    # repeat block
#    for i in range(len(df_word_group_search_repeat1)):        
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)
#
#    try:
#        df_link_default_var = df_link_default
#        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
#        if word_time_diff_var < 4:
#            word_time_diff_var2 = 3
#        else:
#            word_time_diff_var2 = word_time_diff_var+1.0
#
#        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
#    except:
#        pass
#
#    for j in range(3):
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
#        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
#
#df_result_repeat.reset_index(drop=True, inplace=True)
#df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])
#
#df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time2.xlsx", index=False)
#df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join2.xlsx", index=False)

In [65]:
#df_result_repeat

In [66]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [67]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [68]:
df_all_join_result

,video_id,start_time,end_time,search_string,count
0,"ySHVEb0jLFc,yUR3RBa1xP0,jMtqToOsO90,sEhRqPshDc...","134,508,385,562,476,367,271,367,5,367,5,367,5","136,510,386,564,478,369,273,369,8,369,8,369,8",bir şey yok,7.0
1,"qkswPEQJtEY,D4GHc3g6dFw,9htA990fPJI,Qv8LnK8Rsx...","62,417,314,300,253,44,542,417,5,417,5,417,5","63,419,316,302,254,46,544,419,8,419,8,419,8",bir şey değil,7.0
2,"pUUoRExmU1E,Ot0vYkRT1Mg,BO4EFUnpQ48,Q8hqyfbec9...","308,142,676,128,450,529,378,450,5,450,5,450,5","311,144,678,130,452,530,380,452,8,452,8,452,8",şu ana kadar,7.0
3,"tAYKCmpORS0,9htA990fPJI,9htA990fPJI,Vu9P_QeA4K...","158,305,669,15,54,427,34,15,5,15,5,15,5","160,307,671,17,56,429,36,17,8,17,8,17,8",öyle değil mi,7.0
4,"NtoRI5nlwKw,qe8TjfIqZ8Y,uSXzoikCu_k,pLifvTWuvH...","2360,60,9908,8299,3407,2935,5661,60,5,60,5,60,5","2363,62,9910,8301,3409,2937,5664,62,8,62,8,62,8",çok özür dilerim,7.0
5,"OeHxe75lMNE,pZUgbSkK8to,lhms3u5T1m8,UKVile_Hpw...","7449,2723,4221,6688,3134,737,407,3134,5,3134,5...","7451,2725,4223,6690,3136,739,409,3136,8,3136,8...",yok bir şey,7.0
6,"UkA5tTr8JFQ,bKV0xesPfJQ,pWZnIAh9MXo,EAakij0ExE...","1962,6581,6430,553,8000,3696,7731,6430,5,6430,...","1965,6583,6433,555,8002,3698,7733,6433,8,6433,...",tamam o zaman,7.0
7,"z7IjzketUfY,x8yoLCYSzVw,fmpWBnd5VRs,1Nc4d_tmiG...","1799,3426,7966,5902,3939,1278,5050,3426,5,3426...","1801,3428,7968,5904,3942,1280,5053,3428,8,3428...",sen iyi misin,7.0
8,"hbTXsHZBWdI,ZqIIEH7n0L0,A_IMnGvaWtE,xzGVLbiakb...","272,108,75,225,726,328,104,726,5,726,5,726,5","274,109,78,227,728,330,106,728,8,728,8,728,8",neden bu kadar,7.0
9,"zaWgmcdN2JE,v7PglYyVLPQ,liz1ODAVzyk,2ENOaC3O_2...","6597,3969,4140,4220,129,5164,383,5164,5,5164,5...","6598,3971,4142,4222,131,5166,386,5166,8,5166,8...",ne işin var,7.0


#### Copy Move And Delete

In [69]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_1000_Word_Youtube_Talk_Time_Join_Threegram.xlsx']

In [70]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [71]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [43]:
import pandas as pd
import numpy as np

In [45]:
path_folder_file2 = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 2/Result/2-Find Word Group Youtube Link/Turkish_Threegram_With_1000_Word_Selected_Manuel.xlsx"
sheet2 = "Sheet1"  # Sheet1

In [47]:
df = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
df

,search_string,start_time,end_time,sentence,video_id,video_url
0,ben de öyle,267,269,ve böylece kişi kendini şekillendiriyor ben de...,moMs8rXGTjo,https://www.youtube.com/watch?v=moMs8rXGTjo&t=...
1,ben de öyle,5860,5862,i nşallah ben de öyle ümit ediyorum,xIjA_1JQF8I,https://www.youtube.com/watch?v=xIjA_1JQF8I&t=...
2,ben de öyle,52,54,ben de öyle yaptım,4U_xGZWe13Q,https://www.youtube.com/watch?v=4U_xGZWe13Q&t=52s
3,ben de öyle,6569,6572,i yi ben de öyle düşünmüştüm,B2BcTrXcsVE,https://www.youtube.com/watch?v=B2BcTrXcsVE&t=...
4,ben de öyle,5875,5877,i nşallah yavrum ben de öyle düşünüyorum,fI1BI4d5KFU,https://www.youtube.com/watch?v=fI1BI4d5KFU&t=...
...,...,...,...,...,...,...
171,yok bir şey,4221,4223,yok bir şey,lhms3u5T1m8,https://www.youtube.com/watch?v=lhms3u5T1m8&t=...
172,yok bir şey,6688,6690,i yiyim yok bir şey,UKVile_HpwQ,https://www.youtube.com/watch?v=UKVile_HpwQ&t=...
173,yok bir şey,3134,3136,i yiyim iyiyim yok bir şey,kRnKR2D4A0Q,https://www.youtube.com/watch?v=kRnKR2D4A0Q&t=...
174,yok bir şey,737,739,ha yok bir şey ya i yiyim,5R3gWy6I51w,https://www.youtube.com/watch?v=5R3gWy6I51w&t=...


In [48]:
df_count = df.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,bir şey değil,9
1,bir şey yok,9
2,ben de öyle,7
3,ne işin var,7
4,öyle değil mi,7
5,çok özür dilerim,7
6,yok bir şey,7
7,tamam o zaman,7
8,sen iyi misin,7
9,neden bu kadar,7


In [49]:
df_count_merge = pd.merge(df, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,sentence,video_id,video_url,count
0,bir şey yok,453,455,olmaktan spekülasyonlardan yalan haberlerden u...,q3BQl5pv29I,https://www.youtube.com/watch?v=q3BQl5pv29I&t=...,9
1,bir şey yok,101,103,böyle bir şeyleri düşünüyorsun o kadar başka b...,INnAYTwubeo,https://www.youtube.com/watch?v=INnAYTwubeo&t=...,9
2,bir şey yok,134,136,kabul etmekten daha normal de bir şey yok,ySHVEb0jLFc,https://www.youtube.com/watch?v=ySHVEb0jLFc&t=...,9
3,bir şey yok,508,510,öyle birden bire damdan düşer gibi usta olmak ...,yUR3RBa1xP0,https://www.youtube.com/watch?v=yUR3RBa1xP0&t=...,9
4,bir şey yok,385,386,yine burada beklenmedik bir şey yok,jMtqToOsO90,https://www.youtube.com/watch?v=jMtqToOsO90&t=...,9
...,...,...,...,...,...,...,...
171,şaka mı yapıyorsun,1017,1019,şaka mı yapıyorsun gerçekten mi evet maalesef ...,EO4RHGzuFsw,https://www.youtube.com/watch?v=EO4RHGzuFsw&t=...,2
172,beni duyuyor musun,747,749,beni duyuyor musun beni görmen önemli değil,gFzXi6TnQSg,https://www.youtube.com/watch?v=gFzXi6TnQSg&t=...,2
173,beni duyuyor musun,66,69,beni duyuyor musun,VyFnAuumYKk,https://www.youtube.com/watch?v=VyFnAuumYKk&t=66s,2
174,yardım edebilir miyim,1132,1135,ne düşünüyorsunuz yardım edebilir miyim,RqQe0ewBrD0,https://www.youtube.com/watch?v=RqQe0ewBrD0&t=...,1


In [50]:
df_count_merge["search_string"].nunique()

32

In [51]:
df_count_merge[df_count_merge["count"] == 1]

,search_string,start_time,end_time,sentence,video_id,video_url,count
174,yardım edebilir miyim,1132,1135,ne düşünüyorsunuz yardım edebilir miyim,RqQe0ewBrD0,https://www.youtube.com/watch?v=RqQe0ewBrD0&t=...,1
175,senden nefret ediyorum,5436,5438,senden nefret ediyorum ya yerlerinize geçin,JaKhDtaIyvU,https://www.youtube.com/watch?v=JaKhDtaIyvU&t=...,1


In [52]:
df_count_merge.to_excel("Turkish_Threegram_With_1000_Word_Selected_Count_Manuel.xlsx", index=False)

In [ ]:
df_word_group_time_loc_manuel = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Count_Manuel.xlsx")
df_word_group_time_loc_manuel.sort_values(by=["count","search_string"],inplace=True, ascending=False)
df_word_group_time_loc_manuel.reset_index(drop=True, inplace=True)
df_word_group_time_loc_manuel

In [ ]:
df_word_group_time_loc_manuel.to_excel("Twogram Threegram Youtube Link Selected_Manuel.xlsx", index=False)